### Load packages
First we need to load the python packages that are required to perform the analysis

In [ ]:
!pip install salib --upgrade

In [ ]:
import os
import rasterio
import numpy 
import pandas
import scipy.io # to load the matlab files
from SALib.sample import morris
import SALib.analyze.morris
from tqdm import tqdm
import matplotlib.pyplot as plt

from DamageScanner import DamageScanner # the tool to estimate the losses

data_path = os.path.join('..','Data')

numpy.warnings.filterwarnings('ignore')

### Set paths to data and load initial data

All the data is stored in the *Data* folder, so we are creating paths to load data from that folder. We will create a dictionary with the depth-damage information required to do the analysis.

In [ ]:
# function to load DDM data
def read_DDM_input(DDM_data_path):
    DDM_data = {}
    DDM_data['Curves_structure'] = pandas.read_excel(DDM_data_path,sheet_name='curves_structure').values
    DDM_data['Curves_content'] = pandas.read_excel(DDM_data_path,sheet_name='curves_content').values
    DDM_data['MaxDam_structure'] = pandas.read_excel(DDM_data_path,sheet_name='maxdam_structure').values
    DDM_data['MaxDam_content'] = pandas.read_excel(DDM_data_path,sheet_name='maxdam_content').values  
    
    return DDM_data

In [ ]:
inundation_maps = scipy.io.loadmat(os.path.join(data_path,'Waterdepth_current.mat')) # the inundation files for the current situation
landuse_map = os.path.join(data_path,'landuse.asc') # the land-use map
DDM_data = read_DDM_input(os.path.join(data_path,'DDM_data.xlsx'))

### Run the Uncertainty and sensitivity analysis for the structural damages

In [ ]:
 # Set up problem for sensitivity analysis
problem = {
      'num_vars': 3,
      'names': ['inundation', 'maxdam','curves'],
      'bounds': [[XX,XX],[XX,XX],[XX,XX]]} # give reasonable bounds for the different variables that we want to consider

# And create parameter values
param_values = morris.sample(problem, 10, num_levels=5,local_optimization =True)

# Run analysis with the specified parameter values
collect_losses = []
for param_set in tqdm(param_values,total=len(param_values)):
    inundation = inundation_maps['a0_10000'].copy()*param_set[0]
    inundation[inundation>10] = 0
    inundation[inundation<0] = 0

    maxdam = DDM_data['MaxDam_structure'].copy()*param_set[1]

    curves = DDM_data['Curves_structure'].copy()
    curves[:,1:] = curves[:,1:]/param_set[2]

    loss_df = DamageScanner(landuse_map,inundation,curves,maxdam)
    
    collect_losses.append(loss_df[0])

### Obtain results

In [ ]:
Si = SALib.analyze.morris.analyze(problem, numpy.array(param_values), 
                                      numpy.array(collect_losses),
                                 print_to_console=False, num_levels=5)

### And plot the results

In [ ]:
fig = plt.figure(figsize=(10, 5))
ax1 = plt.subplot(121)
ax2 = plt.subplot(122, projection='polar')

color_scheme_map = ['#264653','#2A9D8F']

# Plot boxplot with spread of total losses
ax1.boxplot(numpy.array(collect_losses),showfliers=False, patch_artist=True,
            boxprops=dict(facecolor=color_scheme_map[1], edgecolor='black'),
            capprops=dict(color='black'),
            whiskerprops=dict(color='black'),
            flierprops=dict(color=color_scheme_map[1], markeredgecolor='black'),
            medianprops=dict(color=color_scheme_map[1]),
            )

ax1.set_title('Spread of total damage',fontsize=18,fontweight='black', y=1.08)

# Plot spyder plot with influence of different variables
risk_sens = pandas.DataFrame.from_dict(Si)
risk_sens['rel'] = abs(risk_sens['mu'])/abs(risk_sens['mu']).sum()*100
risk_sens['mu'] = risk_sens['mu']
risk_sens = risk_sens.groupby('names').sum()
risk_sens = risk_sens.T
risk_sens.columns = ['inundation', 'maxdam', 'curve']      

stats=risk_sens.loc['rel',numpy.array(risk_sens.columns)].values

angles=numpy.linspace(0, 2*numpy.pi, len(numpy.array(risk_sens.columns)), endpoint=False)

# close the plot
stats=numpy.concatenate((stats,[stats[0]]))
angles=numpy.concatenate((angles,[angles[0]]))

ax2.plot(angles, stats, 'o-', linewidth=2,color=color_scheme_map[0])
ax2.set_ylim([0, 100])   
ax2.fill(angles, stats, alpha=0.25,color=color_scheme_map[0])
ax2.set_thetagrids(angles * 180/numpy.pi, numpy.array(risk_sens.columns))
ax2.tick_params(axis='x',labelsize=14,labelcolor='black',color='black',) # pad=12

ax2.set_title('Influence of variables',fontsize=18,fontweight='black', y=1.08)

fig.tight_layout()

output_path = os.path.join(data_path,'..','output')
os.mkdir(output_path)
fig.savefig(os.path.join(output_path,'sensitivity_analysis.png'),dpi=300)